In [ ]:
from xtquant import xtdata
def callback (datas):
    print(datas)
xtdata.subscribe_quote(stock_code='600519.SH', period='1m', callback=callback)
xtdata.run()

In [3]:
xtdata.download_sector_data()

In [ ]:
from xtquant import xtdata

def update_progress(progress):
    '''进度条'''
    bar_length = 40  # 进度条长度
    block = int(round(bar_length * progress))
    text = f"\r[{'#' * block + '-' * (bar_length - block)}] {progress * 100:.2f}%"
    if progress < 1:
        print(text, end='', flush=True)
    else:
        print(text, flush=True)


stock_list = xtdata.get_stock_list_in_sector('沪深300')
print(stock_list)
# 实时订阅
def subscribe():
    print('开始订阅')

    for index, stock in enumerate(stock_list):

        def on_data(res, stock=stock):
            print(res, stock)

        xtdata.subscribe_quote(stock_code=stock, period='1d', callback=on_data)
        update_progress((index+1)/len(stock_list))
    
    xtdata.run()

subscribe()

In [ ]:
from xtquant import xtdata
stock_list = xtdata.get_stock_list_in_sector('沪深300')
def callback (datas):
    print(datas)
xtdata.subscribe_whole_quote(code_list=stock_list, callback=callback)
xtdata.run()

In [ ]:
from xtquant import xtdata
import time
def callback (datas):
    print(datas)
seq = xtdata.subscribe_quote(stock_code='600519.SH', period='1d', callback=callback)
print(seq)
time.sleep(10)
res = xtdata.unsubscribe_quote(seq)
print(res)
xtdata.run()


In [ ]:
from xtquant import xtdata
# stock_list = xtdata.get_stock_list_in_sector('沪深300')
res = xtdata.get_full_tick(code_list=['600519.SH'])
res


In [ ]:
from xtquant import xtdata
res = xtdata.get_market_data_ex(stock_list=['600519.SH'], period='1d', start_time='20240101')
res['600519.SH']

In [ ]:
from xtquant import xtdata
import logging
from datetime import datetime
xtdata.enable_hello = False
# =====设置参数=====

period = 'tick' # 周期，可选值 {'tick', '1m', '5m', '1d'}等
stock_list = xtdata.get_stock_list_in_sector('沪深300')

# ====log配置=====

logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# =====主程序=====

def update_progress(progress):
    '''进度条'''
    bar_length = 40  # 进度条长度
    block = int(round(bar_length * progress))
    text = f"\r[{'#' * block + '-' * (bar_length - block)}] {progress * 100:.2f}%"
    if progress < 1:
        print(text, end='', flush=True)
    else:
        print(text, flush=True)

def calculate_seconds_difference(specified_time):
    # 将输入字符串转换为datetime对象
    
    # 获取当前系统时间
    current_time = datetime.now().timestamp()
    
    # 计算时间差
    time_difference =  current_time - (specified_time / 1000)
    
    return time_difference
    

# 实时订阅
def subscribe(stock_list):
    logging.info('开始订阅')

    ready = False

    for index, stock in enumerate(stock_list, start=1):

        def on_data(res, stock=stock):
            if not ready:
                return
            dff = calculate_seconds_difference(res[stock][0]['time'])
            if period != 'tick':
                close_value = res[stock][0]['close']
            else:
                close_value = res[stock][0]['lastPrice']
            # if dff > 3:
            logging.info(f'时间戳：{res[stock][0]["time"]}, 股票代码：{stock}, 当前价格：{close_value}, 延迟：{dff}')

        xtdata.subscribe_quote(stock_code=stock, period=period, count=1, callback=on_data)
        update_progress(index/len(stock_list))
    ready = True
        

def main():
    subscribe(stock_list)
    
if __name__ == "__main__":
    main()
    xtdata.run()
    
